In [3]:
'''
!pip uninstall torch torch-geometric rdkit dpdata rdkit-pypi -y
!pip uninstall pydot dgllife deepchem lightning tf-keras -y
'''
from google.colab import output
output.clear()

In [ ]:
'''
!pip install torch torch-geometric rdkit dpdata rdkit-pypi
!pip install pydot dgllife deepchem lightning
!pip install numpy pyarrow pydot torch tf-keras Pillow
'''

output.clear()

In [2]:
# !pip uninstall tensorflow keras -y
!pip install tensorflow==2.10 keras==2.10 dgl deepchem

output.clear()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation:

In [4]:
import deepchem as dc

In [7]:
from deepchem.feat import ConvMolFeaturizer, WeaveFeaturizer, DMPNNFeaturizer
from deepchem.feat.mol_graphs import ConvMol
import torch
import deepchem as dc
import pandas as pd
import numpy as np
from dgl import DGLGraph
from dgl.nn import GATConv
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dmpnn import DMPNNFeaturizer, DMPNNModel

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


FileNotFoundError: Cannot find DGL C++ graphbolt library at /usr/local/lib/python3.10/dist-packages/dgl/graphbolt/libgraphbolt_pytorch_2.4.0.so

In [6]:
# Load data
train, test = pd.read_csv('/content/train.csv', index_col=False)[['IC50_nM', 'Smiles']], pd.read_csv('/content/test.csv')['Smiles']
X_train, y_train = train['Smiles'], train['IC50_nM']

'''
# Featurize data
conv_X_train, conv_X_test = ConvMolFeaturizer().featurize(X_train), ConvMolFeaturizer().featurize(test)
weave_X_train, weave_X_test = dc.feat.WeaveFeaturizer().featurize(X_train), dc.feat.WeaveFeaturizer().featurize(test)
'''
# DMPNNFeaturizer
dmpnn_X_train, dmpnn_X_test = DMPNNFeaturizer().featurize(X_train), DMPNNFeaturizer().featurize(test)
dmpnn_X_train.shape

NameError: name 'pd' is not defined

In [ ]:
# 전처리: SMILES 문자열을 D-MPNNFeaturizer를 사용하여 피쳐로 변환
featurizer = DMPNNFeaturizer()

# 피쳐와 타겟 변수 추출
X = featurizer.featurize(train['Smiles'])
y = train['IC50_nM'].values

# 데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터를 DGLGraph 형식으로 변환
def create_dgl_graphs(features):
    graphs = []
    for feature in features:
        g = DGLGraph()
        # 여기에서 feature를 기반으로 그래프 삽입
        # (노드와 간선을 추가하는 로직을 구현해야 함)
        graphs.append(g)
    return graphs

train_graphs = create_dgl_graphs(X_train)
test_graphs = create_dgl_graphs(X_test)

# MPNN 모델 정의
class MPNN(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(MPNN, self).__init__()
        self.conv1 = GATConv(in_feats, hidden_size, num_heads=8)
        self.conv2 = GATConv(hidden_size * 8, out_feats, num_heads=1)
        self.fc = nn.Linear(out_feats, 1)

    def forward(self, g, features):
        h = self.conv1(g, features)
        h = self.conv2(g, h)
        readout = dgl.mean_nodes(h)
        return self.fc(readout)

# 모델 초기화
model = MPNN(in_feats=1952, hidden_size=128, out_feats=64)  # your_input_size는 D-MPNNFeaturizer의 출력 크기
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 훈련
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # 각 그래프에 대한 배치 처리
    predictions = []
    for g in train_graphs:
        pred = model(g, g.ndata['features'])
        predictions.append(pred)

    loss = criterion(torch.cat(predictions), torch.tensor(y_train, dtype=torch.float32))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# 테스트
model.eval()
with torch.no_grad():
    test_predictions = []
    for g in test_graphs:
        pred = model(g, g.ndata['features'])
        test_predictions.append(pred)

# 테스트 결과 출력
print("Test Predictions:", test_predictions)